# PyDipAPI - Batch-Operationen und Caching

In diesem Notebook lernen Sie fortgeschrittene Funktionen für effizientes Arbeiten.

## Inhalt:
1. Batch-Anfragen für mehrere IDs
2. Caching aktivieren und nutzen
3. Performance-Optimierung
4. Best Practices

In [1]:
# Setup mit Caching
import sys
import os
import time
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage

# WICHTIG: Ersetzen Sie dies mit Ihrem echten API-Schlüssel!
API_KEY = "OSOegLs.PR2lwJ1dwCeje9vTj7FPOt3hvpYKtwKkhw"

if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    # Client mit aktiviertem Caching
    client = DipAnfrage(
        api_key=API_KEY,
        enable_cache=True,
        cache_ttl=3600,
        rate_limit_delay=0.2
    )
    print("🚀 Client mit Caching initialisiert!")
else:
    print("⚠️  Bitte API-Schlüssel einfügen!")

🚀 Client mit Caching initialisiert!


## Batch-Anfragen

Mehrere IDs auf einmal abfragen:

In [2]:
# Batch-Anfrage demonstrieren
print("📋 Sammle IDs für Batch-Anfragen...")

try:
    # Sammle einige IDs
    docs = client.get_drucksache(anzahl=5)
    doc_ids = [doc.get('id') for doc in docs if doc.get('id')][:3]
    
    print(f"✅ {len(doc_ids)} Dokument-IDs gesammelt: {doc_ids}")
    
    # Batch-Anfrage
    if doc_ids:
        start_time = time.time()
        batch_docs = client.get_drucksache_ids(doc_ids)
        batch_time = time.time() - start_time
        
        print(f"🚀 Batch-Anfrage in {batch_time:.2f}s abgeschlossen")
        print(f"📊 {len(batch_docs)} Dokumente erhalten")
        
except Exception as e:
    print(f"❌ Fehler: {e}")

📋 Sammle IDs für Batch-Anfragen...
✅ 3 Dokument-IDs gesammelt: ['280364', '280362', '280361']
🚀 Batch-Anfrage in 0.02s abgeschlossen
📊 10 Dokumente erhalten


## Performance-Tipps

Wichtige Tipps für optimale Performance:

In [3]:
# Performance-Tipps
print("🏃 Performance-Tipps:")
print("
1. ✅ Batch-Anfragen nutzen:")
print("   client.get_drucksache_ids([1,2,3,4,5])  # Effizient")
print("
2. ✅ Caching aktivieren:")
print("   client = DipAnfrage(api_key=key, enable_cache=True)")
print("
3. ✅ Rate-Limiting beachten:")
print("   client = DipAnfrage(api_key=key, rate_limit_delay=0.2)")
print("
4. ✅ Spezifische Filter verwenden:")
print("   client.search_documents('term', wahlperiode=20, anzahl=50)")
print("
💡 Zusätzliche Tipps:")
print("   • Verwenden Sie angemessene anzahl-Parameter")
print("   • Aktivieren Sie Logging für Debugging")
print("   • Testen Sie mit kleinen Datenmengen zuerst")

SyntaxError: unterminated string literal (detected at line 3) (1586306495.py, line 3)